[Ссылка на задание](https://netology.ru/profile/program/pyda-15/lessons/44588/lesson_items/204455)

*Задание 1*

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [ ]:
#Решение по Заданию 1 (Вариант 1)
import requests
LINK = 'https://www.cbr-xml-daily.ru/daily_json.js'

def get_max_valut():
    rez = list(requests.get(LINK).json()['Valute'].values())
    ret_max = ['', rez[0]['Value'], None]
    for item in rez[1:]:
        if ret_max[1] < item['Value']/item['Nominal']:
            ret_max[0] = item['Name']
            ret_max[1] = item['Value']/item['Nominal']
            ret_max[2] = item['Nominal']

    return ret_max

def get_min_valut():
    rez = list(requests.get(LINK).json()['Valute'].values())
    ret_min = ['', rez[0]['Value'], None]
    for item in rez[1:]:
        if ret_min[1] > item['Value']/item['Nominal']:
            ret_min[0] = item['Name']
            ret_min[1] = item['Value']/item['Nominal']
            ret_min[2] = item['Nominal']
    return ret_min

out = get_max_valut()    
print(f"Максимальный курс - цена {out[2]} '{out[0]}' составляет {out[1]:.4f} руб.")
out = get_min_valut()  
print(f"Минимальный курс - цена {out[2]} '{out[0]}' составляет {out[1]:.4f} руб.")


In [3]:
#Решение по Заданию 1 (Вариант 2)
import requests
r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

data_dict = r.json()['Valute']
max(data_dict.items(), key=lambda x: x[1]['Value'] / x[1]['Nominal'])[1]['Name']

data_dict.items()

dict_items([('AUD', {'ID': 'R01010', 'NumCode': '036', 'CharCode': 'AUD', 'Nominal': 1, 'Name': 'Австралийский доллар', 'Value': 56.0999, 'Previous': 56.5039}), ('AZN', {'ID': 'R01020A', 'NumCode': '944', 'CharCode': 'AZN', 'Nominal': 1, 'Name': 'Азербайджанский манат', 'Value': 45.4311, 'Previous': 46.1777}), ('GBP', {'ID': 'R01035', 'NumCode': '826', 'CharCode': 'GBP', 'Nominal': 1, 'Name': 'Фунт стерлингов Соединенного королевства', 'Value': 101.463, 'Previous': 102.1417}), ('AMD', {'ID': 'R01060', 'NumCode': '051', 'CharCode': 'AMD', 'Nominal': 100, 'Name': 'Армянских драмов', 'Value': 15.6567, 'Previous': 15.893}), ('BYN', {'ID': 'R01090B', 'NumCode': '933', 'CharCode': 'BYN', 'Nominal': 1, 'Name': 'Белорусский рубль', 'Value': 29.9676, 'Previous': 29.9187}), ('BGN', {'ID': 'R01100', 'NumCode': '975', 'CharCode': 'BGN', 'Nominal': 1, 'Name': 'Болгарский лев', 'Value': 46.7293, 'Previous': 47.1773}), ('BRL', {'ID': 'R01115', 'NumCode': '986', 'CharCode': 'BRL', 'Nominal': 1, 'Name'

*Задание 2*

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [82]:
#Решение по Заданию 2 (Вариант 1, наследование + полиморфизм (методы usd ..) + инкапсуляция (метод make_format) )
from libs.exchange import Rate

class RateModiff1(Rate):
    def __init__(self, diff = True):
        super().__init__('value')
        self.diff   = diff

    def make_format(self, currency):
        """
        Возвращает значение 
        """
        response = self.exchange_rates()
        print(self.diff)
        if self.format == 'value': 
            if self.diff:
                return round(response[currency]['Value'] - response[currency]['Previous'], 4)
        return 'Error'

rate_base = Rate()
rate_modif_true  = RateModiff1()
print(f'TEST# Value: {rate_base.usd()}, Course change: {rate_modif_true.usd()}')

True
TEST# Value: 77.1875, Course change: -1.2684


In [83]:
#Решение по Заданию 2 (Вариант 2, новый класс на основе Rate c дополнительным параметром diff)
import requests
class RateModiff2:
    def __init__(self, format_ = 'value', diff = True):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if not self.diff:
                    return response[currency]['Value']
                else:
                    return round(response[currency]['Value'] - response[currency]['Previous'], 4)
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

rate_test1 = RateModiff2('value', True)
rate_test2 = RateModiff2('value', False)
rate_test3 = RateModiff2('full', True)
rate_test4 = RateModiff2('full', False)

print(rate_test1.usd())
print(rate_test2.usd())
print(rate_test3.usd())
print(rate_test4.usd())

-1.2684
77.1875
{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 77.1875, 'Previous': 78.4559}
{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 77.1875, 'Previous': 78.4559}


*Задание 3*

Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [120]:
#Решение по Заданию 3
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority     
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(f'Работник: {self.name}, Грейд: {self.grade}, Баллы: {self.seniority}')
    
    def check_if_it_is_time_for_upgrade(self):
        pass
#-----------------------  
class Designer(Employee):
    def __init__(self, name, seniority = 4):
        super().__init__(name, seniority)
        self.seniority = seniority
    def check_if_it_is_time_for_upgrade(self):
        # для каждой премии увелиичваем счетчик на +2
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 2
        
        # условие повышения сотрудника из презентации
        if (self.seniority - 7*self.grade)  >= 0:
            self.grade_up()
        
        # публикация результатов
        self.publish_grade()

In [126]:
Maria = Designer('Мария')
for i in range(14):
    print(f'Вручение премии {i + 3} !!!')
    Maria.check_if_it_is_time_for_upgrade()

Вручение премии 3!!!
Работник: Мария, Грейд: 1, Баллы: 6
Вручение премии 4!!!
Работник: Мария, Грейд: 2, Баллы: 8
Вручение премии 5!!!
Работник: Мария, Грейд: 2, Баллы: 10
Вручение премии 6!!!
Работник: Мария, Грейд: 2, Баллы: 12
Вручение премии 7!!!
Работник: Мария, Грейд: 3, Баллы: 14
Вручение премии 8!!!
Работник: Мария, Грейд: 3, Баллы: 16
Вручение премии 9!!!
Работник: Мария, Грейд: 3, Баллы: 18
Вручение премии 10!!!
Работник: Мария, Грейд: 3, Баллы: 20
Вручение премии 11!!!
Работник: Мария, Грейд: 4, Баллы: 22
Вручение премии 12!!!
Работник: Мария, Грейд: 4, Баллы: 24
Вручение премии 13!!!
Работник: Мария, Грейд: 4, Баллы: 26
Вручение премии 14!!!
Работник: Мария, Грейд: 5, Баллы: 28
Вручение премии 15!!!
Работник: Мария, Грейд: 5, Баллы: 30
Вручение премии 16!!!
Работник: Мария, Грейд: 5, Баллы: 32
